In [1]:
import pandas as pd
import numpy as np
import re, os

In [2]:

def parse_name(string, df):
    
    string_date = re.findall(r'\d+', string)[0]
    
    if 'lists' in string:
        first_year = string_date[:4]
        first_month = string_date[4:6]
        second_year = string_date[6:10]
        second_month = string_date[10:]
        
        df['first_year'] = first_year
        df['first_month'] = first_month
        df['second_year'] = second_year
        df['second_month'] = second_month
        
    else:
        first_year = string_date[:4]
        first_month = string_date[4:6]
        
        df['year'] = first_year
        df['month'] = first_month
        
    return df

# parse_name('lists201610201705.csv', df_list).head()

### Importing Data

Los datos deben estar guardados en la carpeta _data_ y el cuaderno debe abrirse desde el root del proyecto. Es decir, 

./

 --data (folder)

 --payU_test(notebook)

In [3]:
data_files = os.listdir("data")
data_files = ['data/' + path for path in data_files]

data_files

columns_transactions = ['uuid', 'creation_date', 'email', 'card', 'status', 'chargeback',
                        'amount', 'year', 'month']
data_transactions = pd.DataFrame(columns=columns_transactions)

for file in data_files:
    if 'lists' not in file:
        df_transaction = pd.read_csv(file)
        df_transaction = parse_name(file, df_transaction)
        
        data_transactions = data_transactions.append(df_transaction)
        
    else:
        
        data_lists = pd.read_csv(file)
        data_lists = parse_name(file, data_lists)
        

In [4]:


data_transactions.uuid = data_transactions.uuid.astype(float)
data_lists.uuid = data_lists.uuid.astype(float)

len(data_transactions), len(data_lists)

(721391, 721391)

In [25]:

df_data = pd.merge(data_transactions, data_lists, on='uuid')

df_data.creation_date = pd.to_datetime(df_data.creation_date)
len(df_data)

721467

In [26]:
df_data.head()

,uuid,creation_date,email,card,status,chargeback,amount,year,month,eval_uuid,blacklist,whitelist,first_year,first_month,second_year,second_month
0,-9.063883e+18,2016-10-01 00:00:15,9krrssiithzlag_1a@icloud.com,01EE7EFB10BEDA49E65B84715AB8EF5DDE7E35B4B69F82...,8,False,33.0,2016,10,-7828642419214897884,False,False,2016,10,2017,05
1,-9.206294e+18,2016-10-01 00:00:25,snayyno-ilbeot@hotmail.com,F321F3AD0D64D49262E036A0FDEE7A6EC0D526278888BD...,8,False,10.0,2016,10,-7484788362146786924,False,False,2016,10,2017,05
2,-7.218630e+18,2016-10-01 00:00:41,eeonlar0iodtf@gmail.com,6524F62CFFC63987856643A51D04D15F550C63ACEA9E6B...,8,False,112.0,2016,10,-4693567014900437755,False,False,2016,10,2017,05
3,-8.426635e+18,2016-10-01 00:00:43,snayyno-ilbeot@hotmail.com,F321F3AD0D64D49262E036A0FDEE7A6EC0D526278888BD...,8,False,10.0,2016,10,-8301567106241599310,False,False,2016,10,2017,05
4,-5.727332e+18,2016-10-01 00:00:47,ooyti9vrra8co@outlook.com,A2F9B3ADD703AD4AFEB13E39C9858FC6CAF1622659A43C...,6,False,96.0,2016,10,-6276278361393630476,False,False,2016,10,2017,05


### Duplicated

In [95]:
# print(len(data_lists[data_lists[['uuid']].duplicated()]))
# print(len(data_transactions[data_transactions[['uuid']].duplicated()]))

### Grouping

In [94]:
# df_data.groupby(['year', 'month']).agg({'amount': 'sum'}).sort_values(by = 'amount', ascending = False)
# df_data.groupby(['status']).agg({'amount': 'sum', 'uuid': 'count'}).sort_values(by = 'amount', ascending = False)
# df_data[df_data.status != 8].groupby(['year', 'month', 'status']).agg({'amount': 'sum'}).sort_values(by = 'amount', ascending = False)
# df_data[(df_data.status == 11) | (df_data.status == 12)].groupby(['year', 'month', 'status']).agg({'amount': 'sum'}).sort_values(by = 'amount', ascending = False)
# df_data.groupby(['status', 'chargeback']).agg({'amount': 'sum', 'uuid': 'count'}).sort_values(by = 'amount', ascending = False)

### Transforming

In [27]:
df_data['domain'] = df_data.email.str.split('@').str[1]
df_data['before_domain'] = df_data.email.str.split('@').str[0]
df_data['after_domain'] = df_data.domain.str.split('.', 1).str[1]
df_data['domain'] = df_data.domain.str.split('.', 1).str[0]

df_data.month = df_data.month.astype(int)
df_data.year = df_data.year.astype(int)

# df_data[df_data.status.isin([11,12])].groupby(['month', 'domain'])\
#     .agg({'amount': 'sum', 'uuid': 'count'}).sort_values(by = 'amount', ascending = False)


In [28]:
df_data.head()

,uuid,creation_date,email,card,status,chargeback,amount,year,month,eval_uuid,blacklist,whitelist,first_year,first_month,second_year,second_month,domain,before_domain,after_domain
0,-9.063883e+18,2016-10-01 00:00:15,9krrssiithzlag_1a@icloud.com,01EE7EFB10BEDA49E65B84715AB8EF5DDE7E35B4B69F82...,8,False,33.0,2016,10,-7828642419214897884,False,False,2016,10,2017,05,icloud,9krrssiithzlag_1a,com
1,-9.206294e+18,2016-10-01 00:00:25,snayyno-ilbeot@hotmail.com,F321F3AD0D64D49262E036A0FDEE7A6EC0D526278888BD...,8,False,10.0,2016,10,-7484788362146786924,False,False,2016,10,2017,05,hotmail,snayyno-ilbeot,com
2,-7.218630e+18,2016-10-01 00:00:41,eeonlar0iodtf@gmail.com,6524F62CFFC63987856643A51D04D15F550C63ACEA9E6B...,8,False,112.0,2016,10,-4693567014900437755,False,False,2016,10,2017,05,gmail,eeonlar0iodtf,com
3,-8.426635e+18,2016-10-01 00:00:43,snayyno-ilbeot@hotmail.com,F321F3AD0D64D49262E036A0FDEE7A6EC0D526278888BD...,8,False,10.0,2016,10,-8301567106241599310,False,False,2016,10,2017,05,hotmail,snayyno-ilbeot,com
4,-5.727332e+18,2016-10-01 00:00:47,ooyti9vrra8co@outlook.com,A2F9B3ADD703AD4AFEB13E39C9858FC6CAF1622659A43C...,6,False,96.0,2016,10,-6276278361393630476,False,False,2016,10,2017,05,outlook,ooyti9vrra8co,com


## Análisis exploratorio de los datos

 - El dataset de lists y la unión de los datasets de transacciones, ambos tienen 721391 observaciones. La única columna que comparten es _uuid_ y al realizar un cruce entre ambas tablas, se aumentaron las observaciones en 76 filas. Esto quiere decir que 38 _uuid_ de la tabla lists estaban repetidos y 38 _uuid_ de la tabla de transacciones estaban repetidos.
 
 - El mes que mayor monto en la transacción presenta es marzo del año 2017, seguido de abril. Mientras que el mes que menos monto en la transacción presenta es octubre. Por otro lado el estado _status_ que mayor monto en la transacción presenta es el estado 8. Mientras que las transacciones que se identificaron con estado 11 y 12 son las que menos montos en la transacción presentan
 
 - El único estado _status_ que tiene un chargeback confirmado es el estado 11, cuando efectivamente resultó en un fraude confirmado. 
 
 - Los meses de diciembre, noviembre y octubre son los meses, de mayor a menor, que más número de fraudes confirmados en monto presentan. Mientras que los meses de diciembre, noviembre y febrero son los meses que mayor intento a fraude tienen. Por otro lado los meses de noviembre, marzo y octubre son los meses que más rechazos en las transacciones por alguna entidad presentan en monto.
 
 - Los dominios de correo electrónico que más presentan posibles fraudes _status: 11 y 12_ son por los dominios conocidos como _gmail, hotmail y outlook_. Los dominios que más rechazan las transacciones por alguna de las entidades son _gmail y hotmail_. Sin embargo _gmail y hotmail_ son los dominios que más transacciones realizan en monto. 
 
 - Los dominios _gmail_ y _hotmail_ presentan mayor número de posibles fraudes en los meses octubre, noviembre y diciembre.
 
 
 Cada bullet se analizó con el código comentado en las celdas anteriores, se comentaron porque a pesar de que no toma mucho tiempo su ejecución, solo realiza un display de la tabla a evaluar. Se puede descomentar la línea que se quiera y evaluar en caso tal.

## Preparación de los datos

### Transacción fraudulenta

- Su estado es 11 o 12
- Ha sido marcada como contracargo
- Ha sido incluida en una lista negra

In [29]:
df_data['fraud_status'] = df_data[['status', 'chargeback', 'blacklist']]\
    .apply(lambda df: 1 if df.status in [11, 12] or df.chargeback == 1 or df.blacklist == 1 else 0, axis = 1)

df_data.head()

,uuid,creation_date,email,card,status,chargeback,amount,year,month,eval_uuid,blacklist,whitelist,first_year,first_month,second_year,second_month,domain,before_domain,after_domain,fraud_status
0,-9.063883e+18,2016-10-01 00:00:15,9krrssiithzlag_1a@icloud.com,01EE7EFB10BEDA49E65B84715AB8EF5DDE7E35B4B69F82...,8,False,33.0,2016,10,-7828642419214897884,False,False,2016,10,2017,05,icloud,9krrssiithzlag_1a,com,0
1,-9.206294e+18,2016-10-01 00:00:25,snayyno-ilbeot@hotmail.com,F321F3AD0D64D49262E036A0FDEE7A6EC0D526278888BD...,8,False,10.0,2016,10,-7484788362146786924,False,False,2016,10,2017,05,hotmail,snayyno-ilbeot,com,0
2,-7.218630e+18,2016-10-01 00:00:41,eeonlar0iodtf@gmail.com,6524F62CFFC63987856643A51D04D15F550C63ACEA9E6B...,8,False,112.0,2016,10,-4693567014900437755,False,False,2016,10,2017,05,gmail,eeonlar0iodtf,com,0
3,-8.426635e+18,2016-10-01 00:00:43,snayyno-ilbeot@hotmail.com,F321F3AD0D64D49262E036A0FDEE7A6EC0D526278888BD...,8,False,10.0,2016,10,-8301567106241599310,False,False,2016,10,2017,05,hotmail,snayyno-ilbeot,com,0
4,-5.727332e+18,2016-10-01 00:00:47,ooyti9vrra8co@outlook.com,A2F9B3ADD703AD4AFEB13E39C9858FC6CAF1622659A43C...,6,False,96.0,2016,10,-6276278361393630476,False,False,2016,10,2017,05,outlook,ooyti9vrra8co,com,0



### Transacción fraudulenta

- El email o la tarjeta ha sido usado en una transacción fraudulenta. Este proceso se conoce como expansión de fraudes y se debe continuar haciendo mientras sea posible.


In [30]:
df_emails = df_data.groupby(['email'], as_index=False)\
    .agg({'fraud_status': 'max'}).rename(columns={'fraud_status': 'fraud_status_email'})
    
df_cards = df_data.groupby(['card'], as_index=False)\
    .agg({'fraud_status': 'max'}).rename(columns={'fraud_status': 'fraud_status_card'})
    
df_data = pd.merge(df_data, df_emails, how='left', on='email')
df_data = pd.merge(df_data, df_cards, how='left', on='card')
df_data['fraud'] = df_data[['fraud_status', 'fraud_status_email', 'fraud_status_card']]\
    .apply(lambda df: max(df.fraud_status, df.fraud_status_email, df.fraud_status_card), axis = 1)
    
df_data.head()

,uuid,creation_date,email,card,status,chargeback,amount,year,month,eval_uuid,...,first_month,second_year,second_month,domain,before_domain,after_domain,fraud_status,fraud_status_email,fraud_status_card,fraud
0,-9.063883e+18,2016-10-01 00:00:15,9krrssiithzlag_1a@icloud.com,01EE7EFB10BEDA49E65B84715AB8EF5DDE7E35B4B69F82...,8,False,33.0,2016,10,-7828642419214897884,...,10,2017,05,icloud,9krrssiithzlag_1a,com,0,0,0,0
1,-9.206294e+18,2016-10-01 00:00:25,snayyno-ilbeot@hotmail.com,F321F3AD0D64D49262E036A0FDEE7A6EC0D526278888BD...,8,False,10.0,2016,10,-7484788362146786924,...,10,2017,05,hotmail,snayyno-ilbeot,com,0,0,0,0
2,-7.218630e+18,2016-10-01 00:00:41,eeonlar0iodtf@gmail.com,6524F62CFFC63987856643A51D04D15F550C63ACEA9E6B...,8,False,112.0,2016,10,-4693567014900437755,...,10,2017,05,gmail,eeonlar0iodtf,com,0,0,0,0
3,-8.426635e+18,2016-10-01 00:00:43,snayyno-ilbeot@hotmail.com,F321F3AD0D64D49262E036A0FDEE7A6EC0D526278888BD...,8,False,10.0,2016,10,-8301567106241599310,...,10,2017,05,hotmail,snayyno-ilbeot,com,0,0,0,0
4,-5.727332e+18,2016-10-01 00:00:47,ooyti9vrra8co@outlook.com,A2F9B3ADD703AD4AFEB13E39C9858FC6CAF1622659A43C...,6,False,96.0,2016,10,-6276278361393630476,...,10,2017,05,outlook,ooyti9vrra8co,com,0,0,0,0


### Transacciones legítimas

- Las transacciones con estado 8 que no fueron marcadas por la norma 1 como fraudulentas, son consideradas legítimas.

In [31]:
df_data['tr_legit'] = df_data[['status', 'fraud']].apply(lambda df: 1 if df.status == 8 and df.fraud == 0 else 0, axis = 1)
df_data

,uuid,creation_date,email,card,status,chargeback,amount,year,month,eval_uuid,...,second_year,second_month,domain,before_domain,after_domain,fraud_status,fraud_status_email,fraud_status_card,fraud,tr_legit
0,-9.063883e+18,2016-10-01 00:00:15,9krrssiithzlag_1a@icloud.com,01EE7EFB10BEDA49E65B84715AB8EF5DDE7E35B4B69F82...,8,False,33.0,2016,10,-7828642419214897884,...,2017,05,icloud,9krrssiithzlag_1a,com,0,0,0,0,1
1,-9.206294e+18,2016-10-01 00:00:25,snayyno-ilbeot@hotmail.com,F321F3AD0D64D49262E036A0FDEE7A6EC0D526278888BD...,8,False,10.0,2016,10,-7484788362146786924,...,2017,05,hotmail,snayyno-ilbeot,com,0,0,0,0,1
2,-7.218630e+18,2016-10-01 00:00:41,eeonlar0iodtf@gmail.com,6524F62CFFC63987856643A51D04D15F550C63ACEA9E6B...,8,False,112.0,2016,10,-4693567014900437755,...,2017,05,gmail,eeonlar0iodtf,com,0,0,0,0,1
3,-8.426635e+18,2016-10-01 00:00:43,snayyno-ilbeot@hotmail.com,F321F3AD0D64D49262E036A0FDEE7A6EC0D526278888BD...,8,False,10.0,2016,10,-8301567106241599310,...,2017,05,hotmail,snayyno-ilbeot,com,0,0,0,0,1
4,-5.727332e+18,2016-10-01 00:00:47,ooyti9vrra8co@outlook.com,A2F9B3ADD703AD4AFEB13E39C9858FC6CAF1622659A43C...,6,False,96.0,2016,10,-6276278361393630476,...,2017,05,outlook,ooyti9vrra8co,com,0,0,0,0,0
5,-6.685762e+18,2016-10-01 00:00:48,nrypaea@hotmail.com,503079B977E3032B7707F2AC03B328DFD8A718336AB442...,6,False,10.0,2016,10,-8731713242758708783,...,2017,05,hotmail,nrypaea,com,0,0,0,0,0
6,-7.223667e+18,2016-10-01 00:01:05,istoaprl.leeny@gmail.com,E19F04BBB30089869D76824BB283D87F50E96F416D7338...,8,False,133.0,2016,10,-7713952596083430419,...,2017,05,gmail,istoaprl.leeny,com,0,0,0,0,1
7,-6.310181e+18,2016-10-01 00:01:32,nrypaea@hotmail.com,503079B977E3032B7707F2AC03B328DFD8A718336AB442...,6,False,10.0,2016,10,-4874212705000137974,...,2017,05,hotmail,nrypaea,com,0,0,0,0,0
8,-5.828803e+18,2016-10-01 00:01:52,ivm_aaaple@yahoo.com,EA7A5C5CF1644E003B673479992CD79B091F960C4AA5BB...,8,False,10.0,2016,10,-7661922811906668630,...,2017,05,yahoo,ivm_aaaple,com,0,0,0,0,1
9,-7.628880e+18,2016-10-01 00:02:41,rrznmriaienbaz_@outlook.com,832D40E9203C5272B61D1B2F70E1F4A314F2A23F1BD014...,8,False,10.0,2016,10,-7048153610263768680,...,2017,05,outlook,rrznmriaienbaz_,com,0,0,0,0,1


### Transacciones eliminadas

- Todas las transacciones que no hayan sido tenidas en cuenta por las normas 1 y 2 deben ser eliminadas del dataset.

In [150]:
df_processed = df_data[(df_data.tr_legit == 1) | (df_data.fraud == 1)]
print('Data inicial: ' + str(len(df_data)))
print('Data procesada: ' + str(len(df_processed)))

df_processed.head()

Data inicial: 721467
Data procesada: 507577


,uuid,creation_date,email,card,status,chargeback,amount,year,month,eval_uuid,...,second_year,second_month,domain,before_domain,after_domain,fraud_status,fraud_status_email,fraud_status_card,fraud,tr_legit
0,-9.063883e+18,2016-10-01 00:00:15,9krrssiithzlag_1a@icloud.com,01EE7EFB10BEDA49E65B84715AB8EF5DDE7E35B4B69F82...,8,False,33.0,2016,10,-7828642419214897884,...,2017,05,icloud,9krrssiithzlag_1a,com,0,0,0,0,1
1,-9.206294e+18,2016-10-01 00:00:25,snayyno-ilbeot@hotmail.com,F321F3AD0D64D49262E036A0FDEE7A6EC0D526278888BD...,8,False,10.0,2016,10,-7484788362146786924,...,2017,05,hotmail,snayyno-ilbeot,com,0,0,0,0,1
2,-7.218630e+18,2016-10-01 00:00:41,eeonlar0iodtf@gmail.com,6524F62CFFC63987856643A51D04D15F550C63ACEA9E6B...,8,False,112.0,2016,10,-4693567014900437755,...,2017,05,gmail,eeonlar0iodtf,com,0,0,0,0,1
3,-8.426635e+18,2016-10-01 00:00:43,snayyno-ilbeot@hotmail.com,F321F3AD0D64D49262E036A0FDEE7A6EC0D526278888BD...,8,False,10.0,2016,10,-8301567106241599310,...,2017,05,hotmail,snayyno-ilbeot,com,0,0,0,0,1
6,-7.223667e+18,2016-10-01 00:01:05,istoaprl.leeny@gmail.com,E19F04BBB30089869D76824BB283D87F50E96F416D7338...,8,False,133.0,2016,10,-7713952596083430419,...,2017,05,gmail,istoaprl.leeny,com,0,0,0,0,1


### Feature Engineering

- Agregue las características que considere que pueden ser útiles en la generación de modelos de clasificación para identificar transacciones fraudulentas.

In [151]:
df_model = df_processed.groupby(['email', 'card', 'status'], as_index = False)\
    .agg({'uuid': 'count', 'chargeback': 'max', 
          'blacklist': 'max', 'amount': ['sum', 'mean', 'max', 'min'],
          'year': ['nunique'], 
          'month': ['nunique'], #'domain': 'min','after_domain': 'min',
           'whitelist': 'max', 'fraud_status': 'max', 'fraud_status_email': 'max',
          'fraud_status_card': 'max', 'fraud': 'max', 'tr_legit': 'max'}, as_index = False)



df_model.columns = list(map(''.join, df_model.columns.values))
df_model.head()

,email,card,status,uuidcount,chargebackmax,fraud_status_emailmax,fraud_status_cardmax,blacklistmax,yearnunique,whitelistmax,monthnunique,fraudmax,amountsum,amountmean,amountmax,amountmin,fraud_statusmax,tr_legitmax
0,+691df3rpod@gmail.com,7015033AC9CD7DD426EEFAE7D59411B232D830FB1ADC78...,8,1,False,0,0,False,1,False,1,0,10.0,10.000000,10.0,10.0,0,1
1,+azol.yeaan1p@easytaxi.com.mx,452BD6B46DB2BAF5F60D132DEE69535739C8D3C821E75A...,8,10,False,0,0,False,2,False,3,0,100.0,10.000000,10.0,10.0,0,1
2,+bsscirmaoau1@gmail.com,3ED1E7766B594FAA63CDD461F3929E1C6F362F7C3AD864...,8,1,False,0,0,False,1,False,1,0,10.0,10.000000,10.0,10.0,0,1
3,+cg1assa@websinergia.com.mx,11F22A2A91ED019168DA76370C0462672B573241E20566...,8,3,False,0,0,False,1,False,1,0,220.0,73.333333,200.0,10.0,0,1
4,--nar9p7om@hotmail.com,6FAD6147FA91B8C4163157208240173CAC22B8279B28AF...,8,2,False,0,0,False,1,False,1,0,20.0,10.000000,10.0,10.0,0,1


### Feature Engineering

- Al menos dos características que analicen el texto del correo electrónico (pueden ser características simples como la longitud de la cadena, o el número de caracteres numéricos)

Además de estas se calcularon otras variables como dominio del correo, caracteres después del dominio, etc.

In [152]:
df_processed['digits_email'] = df_processed.before_domain\
    .apply(lambda x: np.sum(([len(s) for s in re.findall(r'-?\d+\.?\d*', x)])))
    
df_processed['len_email'] = df_processed.before_domain\
    .apply(lambda x: len(x))

    
df_email_features = df_processed.groupby(['email', 'card', 'status'], as_index = False)\
    .agg({'digits_email': 'sum', 'len_email': 'sum'})
    
df_email_features.columns = list(map(''.join, df_email_features.columns.values))
df_email_features.head()

C:\Python35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Python35\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,email,card,status,len_email,digits_email
0,+691df3rpod@gmail.com,7015033AC9CD7DD426EEFAE7D59411B232D830FB1ADC78...,8,11,4.0
1,+azol.yeaan1p@easytaxi.com.mx,452BD6B46DB2BAF5F60D132DEE69535739C8D3C821E75A...,8,130,10.0
2,+bsscirmaoau1@gmail.com,3ED1E7766B594FAA63CDD461F3929E1C6F362F7C3AD864...,8,13,1.0
3,+cg1assa@websinergia.com.mx,11F22A2A91ED019168DA76370C0462672B573241E20566...,8,24,3.0
4,--nar9p7om@hotmail.com,6FAD6147FA91B8C4163157208240173CAC22B8279B28AF...,8,20,4.0


### Feature Engineering

- Una característica que contenga el número de veces que el email de la transacción ha sido visto en el pasado (antes de su fecha de creación).

In [153]:
def prior_check_create_date(df):
    
    dates_prior = np.sum([1 if cr_date > min_cr_date else 0 \
                         for cr_date, min_cr_date in zip(df.creation_date, df.min_creation_date)])
    return pd.Series(dict(dates_prior = dates_prior))
    
    
df_creation_date = df_processed.groupby(['email', 'card'], as_index=False)\
    .agg({'creation_date': 'min'}).rename(columns={'creation_date': 'min_creation_date'})

df_processed = pd.merge(df_processed, df_creation_date, how='left', on=['email', 'card'])

df_creation_date_status = df_processed.groupby(['email', 'card', 'status'], as_index = False)\
    .apply(prior_check_create_date).reset_index()
    
df_creation_date_status.columns = list(map(''.join, df_creation_date_status.columns.values))

df_creation_date_status.head()

,email,card,status,dates_prior
0,+691df3rpod@gmail.com,7015033AC9CD7DD426EEFAE7D59411B232D830FB1ADC78...,8,0
1,+azol.yeaan1p@easytaxi.com.mx,452BD6B46DB2BAF5F60D132DEE69535739C8D3C821E75A...,8,9
2,+bsscirmaoau1@gmail.com,3ED1E7766B594FAA63CDD461F3929E1C6F362F7C3AD864...,8,0
3,+cg1assa@websinergia.com.mx,11F22A2A91ED019168DA76370C0462672B573241E20566...,8,2
4,--nar9p7om@hotmail.com,6FAD6147FA91B8C4163157208240173CAC22B8279B28AF...,8,1


In [154]:
df_model = pd.merge(df_model, df_creation_date_status, on=['email', 'card', 'status'])

df_model = pd.merge(df_model, df_email_features, on=['email', 'card', 'status'])

df_model.head()

,email,card,status,uuidcount,chargebackmax,fraud_status_emailmax,fraud_status_cardmax,blacklistmax,yearnunique,whitelistmax,...,fraudmax,amountsum,amountmean,amountmax,amountmin,fraud_statusmax,tr_legitmax,dates_prior,len_email,digits_email
0,+691df3rpod@gmail.com,7015033AC9CD7DD426EEFAE7D59411B232D830FB1ADC78...,8,1,False,0,0,False,1,False,...,0,10.0,10.000000,10.0,10.0,0,1,0,11,4.0
1,+azol.yeaan1p@easytaxi.com.mx,452BD6B46DB2BAF5F60D132DEE69535739C8D3C821E75A...,8,10,False,0,0,False,2,False,...,0,100.0,10.000000,10.0,10.0,0,1,9,130,10.0
2,+bsscirmaoau1@gmail.com,3ED1E7766B594FAA63CDD461F3929E1C6F362F7C3AD864...,8,1,False,0,0,False,1,False,...,0,10.0,10.000000,10.0,10.0,0,1,0,13,1.0
3,+cg1assa@websinergia.com.mx,11F22A2A91ED019168DA76370C0462672B573241E20566...,8,3,False,0,0,False,1,False,...,0,220.0,73.333333,200.0,10.0,0,1,2,24,3.0
4,--nar9p7om@hotmail.com,6FAD6147FA91B8C4163157208240173CAC22B8279B28AF...,8,2,False,0,0,False,1,False,...,0,20.0,10.000000,10.0,10.0,0,1,1,20,4.0


In [155]:
df_model.columns

Index(['email', 'card', 'status', 'uuidcount', 'chargebackmax',
       'fraud_status_emailmax', 'fraud_status_cardmax', 'blacklistmax',
       'yearnunique', 'whitelistmax', 'monthnunique', 'fraudmax', 'amountsum',
       'amountmean', 'amountmax', 'amountmin', 'fraud_statusmax',
       'tr_legitmax', 'dates_prior', 'len_email', 'digits_email'],
      dtype='object')

In [156]:
#Status as categorical
df_model = pd.concat([df_model, pd.get_dummies(df_model.status, prefix='status')], axis = 1)

# #domain as categorical
# df_model = pd.concat([df_model, pd.get_dummies(df_model.domainmin, prefix='domain')], axis = 1)

# #after_domain as categorical
# df_model = pd.concat([df_model, pd.get_dummies(df_model.after_domainmin, prefix='after_domain')], axis = 1)

#removing the categorical variables
df_model = df_model.drop(['status'], axis = 1)#, 'domainmin', 'after_domainmin'], axis = 1)
df_model.head()

,email,card,uuidcount,chargebackmax,fraud_status_emailmax,fraud_status_cardmax,blacklistmax,yearnunique,whitelistmax,monthnunique,...,dates_prior,len_email,digits_email,status_3,status_4,status_6,status_8,status_11,status_12,status_13
0,+691df3rpod@gmail.com,7015033AC9CD7DD426EEFAE7D59411B232D830FB1ADC78...,1,False,0,0,False,1,False,1,...,0,11,4.0,0,0,0,1,0,0,0
1,+azol.yeaan1p@easytaxi.com.mx,452BD6B46DB2BAF5F60D132DEE69535739C8D3C821E75A...,10,False,0,0,False,2,False,3,...,9,130,10.0,0,0,0,1,0,0,0
2,+bsscirmaoau1@gmail.com,3ED1E7766B594FAA63CDD461F3929E1C6F362F7C3AD864...,1,False,0,0,False,1,False,1,...,0,13,1.0,0,0,0,1,0,0,0
3,+cg1assa@websinergia.com.mx,11F22A2A91ED019168DA76370C0462672B573241E20566...,3,False,0,0,False,1,False,1,...,2,24,3.0,0,0,0,1,0,0,0
4,--nar9p7om@hotmail.com,6FAD6147FA91B8C4163157208240173CAC22B8279B28AF...,2,False,0,0,False,1,False,1,...,1,20,4.0,0,0,0,1,0,0,0


In [163]:
print('modelo')
print('... ')
print('Número de cols: ' + str(len(df_model.columns)))
print('Número de observaciones: ' + str(len(df_model)))
print('...')
df_model.dtypes

modelo
... 
Número de cols: 27
Número de observaciones: 77477
...


email                     object
card                      object
uuidcount                  int64
chargebackmax               bool
fraud_status_emailmax      int64
fraud_status_cardmax       int64
blacklistmax                bool
yearnunique                int64
whitelistmax                bool
monthnunique               int64
fraudmax                   int64
amountsum                float64
amountmean               float64
amountmax                float64
amountmin                float64
fraud_statusmax            int64
tr_legitmax                int64
dates_prior                int64
len_email                  int64
digits_email             float64
status_3                   uint8
status_4                   uint8
status_6                   uint8
status_8                   uint8
status_11                  uint8
status_12                  uint8
status_13                  uint8
dtype: object

### Entrenamiento


In [227]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import  preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split

objective = 'fraudmax'
ids = ['email', 'card']
fraud_var = ['fraud_status_cardmax', 'fraud_status_emailmax', 'tr_legitmax', 'fraud_statusmax']

y = df_model[objective]
X = df_model[list(set(list(df_model.columns)) - set(list([objective]) + list(ids) + list(fraud_var)))]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = True)

print('data entrenamiento')
print('... ')
print('Número de cols: ' + str(len(X_train.columns)))
print('Número de observaciones: ' + str(len(X_train)))
print('...')


print('data test')
print('... ')
print('Número de cols: ' + str(len(X_test.columns)))
print('Número de observaciones: ' + str(len(X_test)))
print('...')

data entrenamiento
... 
Número de cols: 20
Número de observaciones: 54233
...
data test
... 
Número de cols: 20
Número de observaciones: 23244
...


In [229]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [230]:
y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.97074513853


In [231]:
clf2 = DecisionTreeClassifier()
clf2.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [232]:
y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.97074513853


- ¿Qué modelos escogió?

Los modelos escogidos fueron DecisionTreeClassifier y RandomForestClassifier. Ambos métodos muy potentes para técnicas donde interactúan muchas variables. Uno, un método de ensamble y otro un método de clasificación de árboles. La escogencia de estos modelos fue al azar entre algunas otras opciones con las que había trabajado antes. 

- ¿Cómo hizo la partición?

La partición de los datos se hizo mediante un _train test split_ porque esta función parte desde la premisa que no existen datos de validación y se encarga de partir los datos de entrenamiento entre, datos de training y testing (si se quiere se puede agregar un nivel adicional que son los datos de validating, éstos más complejos). Se quiso configurar los modelos con lo básico que trae por defecto, para no intervenir en un overfitting. La escogencia de los metaparámetros se puede realizar para empezar a mejorar un poco el modelo y eventualmente recalibrarlo. 

- ¿Cuáles fueron las variables más importantes para los clasificadores que construyó?

Las variables más importantes para los modelos están descritas en las siguietnes dos celdas. La primera de ellas para el modelo de RandomForestClassifier y las segunda para el modelo DecisionTreesClassifier. Al parecer, fuera de las variables de estado, otra que figura ser muy importante es la longitud del email, junto con el monto de la transacción.



In [235]:
feature_importances = pd.DataFrame(clf.feature_importances_,
                                   index = X_train.columns,
                                   columns=['importance'])\
    .sort_values('importance', ascending=False)

print('RandomForestClassifier')
print()
feature_importances

RandomForestClassifier



,importance
status_8,0.568535
status_12,0.092522
status_4,0.063438
status_11,0.042721
amountmean,0.031070
len_email,0.028134
blacklistmax,0.027002
amountsum,0.023445
amountmax,0.022702
status_6,0.018596


In [236]:
feature_importances = pd.DataFrame(clf2.feature_importances_,
                                   index = X_train.columns,
                                   columns=['importance'])\
    .sort_values('importance', ascending=False)

print('DecisionTreesClassifier')
print()
feature_importances

DecisionTreesClassifier



,importance
status_8,0.853698
len_email,0.026813
amountmean,0.026234
amountmax,0.023449
amountsum,0.023149
dates_prior,0.014522
digits_email,0.011139
uuidcount,0.010172
monthnunique,0.005201
blacklistmax,0.003939


- ¿Tuvo limitaciones para la creación de los modelos? ¿Cuáles?

Sobre todo para la parte del entendimiento de los datos. Cómo eran y cómo podían comunicarse las tablas. El diccionario de datos que se dio, fue muy útil para el entendimiento de las etiquetas a los largo de las variables.


- ¿Qué mejoraría en los datos para obtener resultados superiores?

La verdad no quise entrar a jugar con los híper parámetros por cuestiones de tiempo. Pero eso es un tema muy importante a la hora de mejorar el rendimiento de un modelo. Adicionalmente me gusta gastar mucho más tiempo en la parte del data cleansing y feature engineering, me parece que ahí es donde está, posiblemente, todo el poder. Una vez agotado esto, se podría jugar con las interacciones entre variables y las variables categóricas. Por último jugar entre modelos y desechar las varaibles que no aporten información al modelo, mediante técnicas para hacer reducción de dimensionalidad.


- ¿Cómo pondría en operación los modelos que construyó? ¿Cómo seleccionaría los umbrales de decisión para clasificar una transacción nueva como legítima o fraudulenta?

Bueno, la verdad esto es un dataset muy pequeño y al tener un dataset gigante entraríamos en problemas de recursos físicos y digitales. Dependiendo de lo que se quiera, está la solución. Pero por lo general se debe entrenar los modelos con cierta recurrencia y guardar un pickle, pueden ser otros formatos, para que sea alimentado con datos que el modelo nunca ha tocado y de esta manera pueda entregar una clasificación. Los umbrales pueden ser dinámicos y eso es otro campo de estudio bastante complejo. Pero al ser esto un problema clasificatorio, también se puede jugar un poco con la probabilidad que un modelo 'predice' una clase a la cual pertenece una observación. Mirar cuáles son las causas cuando ocurre una probabilidad casi certera y cuándo ocurre una ambigüedad donde el modelo no tiene suficiente información para decidir. Otra estrategia es jugar con algún punto (o algunos) de la matriz de confusión y concentrándose en solo ese problema.